In [1]:
%matplotlib inline

In [35]:
import pandas as pd
from collections import Counter
from Bio import Phylo
import copy

# Higher order summary statistics of growth rates dataset

**Read in dataset**

In [ ]:
growth_df = pd.read_csv('../Data/growth_rate_dataset_10_2017.csv', delimiter='\t', index_col='#Id')
print(len(growth_df.index))

**Get rid of Archaea**

In [ ]:
growth_df = growth_df[growth_df['Full Taxonomy'].str.startswith('Bacteria')==True].copy()
print(len(growth_df.index))

In [ ]:
growth_df.head()

**Calculate a few of the new statistics from the previous columns**

In [ ]:
###aSD binding score is the ribosomal protein score relative to all genes
growth_df['aSD_binding_summary'] = (growth_df['aSD_binding_score_ribosome'] - growth_df['aSD_binding_score_all']) / growth_df['aSD_binding_score_all']

growth_df['folding_energy_init_summary'] = (growth_df['Mean_folding_init_ribo'] - growth_df['Mean_folding_init'])/ growth_df['Mean_folding_init']
growth_df['folding_energy_diff_summary'] = (growth_df['Mean_folding_diff_ribo'] - growth_df['Mean_folding_diff'])/ growth_df['Mean_folding_diff']

###Codon usage bias score is the ribosomal protein score relative to all genes
###Using two methods that perform well (ours=info, novembre=method developed by John Novembre paper)
growth_df['NC_summary_info'] = (growth_df['Ribo_NC_info'] - growth_df['All_NC_info']) / growth_df['All_NC_info']
growth_df['NC_summary_novembre'] = (growth_df['Ribo_NC_novembre'] - growth_df['All_NC_novembre']) / growth_df['All_NC_novembre']

###Delta I score for all genes and ribosomal protein coding genes
growth_df['SD_summary_all_info'] = growth_df['Information_content_all(-20_-4)'] - growth_df['Information_content_all_mean_random(-20_-4)']
growth_df['SD_summary_ribo_info'] = growth_df['Information_content_ribo(-20_-4)'] - growth_df['Information_content_ribo_mean_random(-20_-4)']

###Summary score for "Traditional" SD methods, SD gene fraction minus expected SD gene fraction
growth_df['SD_summary_all_motif_diff'] = (growth_df['CDS_with_SD_utr_motif']/growth_df['CDS_number']) - (growth_df['Expected_CDS_with_SD_utr_motif']/growth_df['CDS_number'])
growth_df['SD_summary_all_binding_diff'] = (growth_df['CDS_with_SD_utr_binding']/growth_df['CDS_number']) - (growth_df['Expected_CDS_with_SD_utr_binding']/growth_df['CDS_number'])

**Write new file**

In [ ]:
growth_df.to_csv('../Data/growth_rate_dataset_10_2017_EXTENDED.csv', sep='\t')

**And a separate file without Bacteroidetes**

In [ ]:
growth_df_no_bacteroidetes = growth_df[growth_df['Full Taxonomy'].str.contains('acteroidetes')==False].copy()
print(len(growth_df_no_bacteroidetes.index))

In [ ]:
growth_df_no_bacteroidetes.to_csv('../Data/growth_rate_dataset_10_2017_EXTENDED_NO_BACTEROIDETES.csv', sep='\t')

# Higher order summary statistics for larger dataset

**Read in data**

In [17]:
phylo_df = pd.read_table('../Data/full_dataset_revisions_10_2017.csv', sep='\t', index_col='#Id')

###For some reason, people make columns with spaces which is of course a debacle that must be rectified
if sum([c.count(' ') for c in phylo_df.columns]) != 0:
    phylo_df.columns = [c.replace(' ', '_') for c in phylo_df.columns]
    print("had columns with spaces")

had columns with spaces


**Manually inspect and drop some data**

In [18]:
###Any one with realllllly tiny genomes?
phylo_df[phylo_df['CDS_number'] <= 500]

,Organism,TaxonID,Full_Taxonomy,Banfield_tree_name,CDS_GC,CDS_GC_nostartstop,Ribo_prot_GC_nostartstop,CDS_number,Intragenic_GC,Gene_density,...,RP_information_zscore,aSD_binding_score_ribosome,aSD_binding_score_all,Internal_SD_seqs,Mean_folding_init,Mean_folding_internal,Mean_folding_diff,Mean_folding_init_ribo,Mean_folding_internal_ribo,Mean_folding_diff_ribo
#Id,,,,,,,,,,,,,,,,,,,,,
GCA_000093065,Candidatus Riesia pediculicola USDA,515618,Bacteria; Proteobacteria; Gammaproteobacteria;...,Bacteria_Proteobacteria_Gammaproteobacteria_En...,0.296054,0.296395,0.305903,464.0,0.208607,0.820392,...,1.399164,2.608778,2.858529,177.0,-4.528621,-6.543405,2.014784,-4.211538,-6.928846,2.717308
GCA_000219175,Candidatus Moranella endobia PCIT,903503,Bacteria; Proteobacteria; Gammaproteobacteria;...,Bacteria_Proteobacteria_Gammaproteobacteria_En...,0.452321,0.453570,0.450272,411.0,0.434766,0.807620,...,3.374380,1.808119,2.339969,47.0,-7.323961,-11.166846,3.842885,-7.646154,-11.583462,3.937308
GCA_000441555,Candidatus Profftella armatura,669502,Bacteria; Proteobacteria; Betaproteobacteria; ...,Bacteria_Proteobacteria_Betaproteobacteria_unc...,0.252415,0.252791,0.283813,347.0,0.119577,0.892542,...,-0.386534,2.182660,2.343993,67.0,-3.430174,-5.978783,2.548609,-3.878846,-6.188462,2.309615


In [19]:
###Anyone that doesn't have ribosomes? Because that's kind of bad
phylo_df[phylo_df['16s_copies'] == 0]

,Organism,TaxonID,Full_Taxonomy,Banfield_tree_name,CDS_GC,CDS_GC_nostartstop,Ribo_prot_GC_nostartstop,CDS_number,Intragenic_GC,Gene_density,...,RP_information_zscore,aSD_binding_score_ribosome,aSD_binding_score_all,Internal_SD_seqs,Mean_folding_init,Mean_folding_internal,Mean_folding_diff,Mean_folding_init_ribo,Mean_folding_internal_ribo,Mean_folding_diff_ribo
#Id,,,,,,,,,,,,,,,,,,,,,
GCA_000210735,Faecalibacterium prausnitzii L2-6,718252,Bacteria; Firmicutes; Clostridia; Clostridiale...,Bacteria_Firmicutes_Clostridia_Clostridiales_R...,0.585890,0.588214,0.594929,2756.0,0.527245,0.716427,...,5.831106,2.537646,3.342250,849.0,-9.067978,-16.141271,7.073293,-7.837736,-15.973585,8.135849
GCA_000209815,Butyrivibrio fibrisolvens 16/4,657324,Bacteria; Firmicutes; Clostridia; Clostridiale...,Bacteria_Firmicutes_Clostridia_Clostridiales_L...,0.392122,0.393274,0.427456,2904.0,0.362914,0.758636,...,4.089214,2.358450,2.975419,717.0,-6.212705,-9.871553,3.658848,-6.308000,-11.168000,4.860000


In [20]:
###Any truly ridiculous GC contents?
phylo_df[(phylo_df['CDS_GC'] < 0.2) | (phylo_df['CDS_GC'] > 0.8)]

,Organism,TaxonID,Full_Taxonomy,Banfield_tree_name,CDS_GC,CDS_GC_nostartstop,Ribo_prot_GC_nostartstop,CDS_number,Intragenic_GC,Gene_density,...,RP_information_zscore,aSD_binding_score_ribosome,aSD_binding_score_all,Internal_SD_seqs,Mean_folding_init,Mean_folding_internal,Mean_folding_diff,Mean_folding_init_ribo,Mean_folding_internal_ribo,Mean_folding_diff_ribo
#Id,,,,,,,,,,,,,,,,,,,,,


In [21]:
###Any with ridiculously high/low coding sequence density calculations?
phylo_df[(phylo_df['Coding_density'] < 0.5) | (phylo_df['Coding_density'] > 0.99)]

,Organism,TaxonID,Full_Taxonomy,Banfield_tree_name,CDS_GC,CDS_GC_nostartstop,Ribo_prot_GC_nostartstop,CDS_number,Intragenic_GC,Gene_density,...,RP_information_zscore,aSD_binding_score_ribosome,aSD_binding_score_all,Internal_SD_seqs,Mean_folding_init,Mean_folding_internal,Mean_folding_diff,Mean_folding_init_ribo,Mean_folding_internal_ribo,Mean_folding_diff_ribo
#Id,,,,,,,,,,,,,,,,,,,,,


In [22]:
###Any with ridiculously high / low nubmers of ribosomal protein coding genes?
phylo_df[(phylo_df['Ribo_prot_number'] < 30) |(phylo_df['Ribo_prot_number'] > 100) ]

,Organism,TaxonID,Full_Taxonomy,Banfield_tree_name,CDS_GC,CDS_GC_nostartstop,Ribo_prot_GC_nostartstop,CDS_number,Intragenic_GC,Gene_density,...,RP_information_zscore,aSD_binding_score_ribosome,aSD_binding_score_all,Internal_SD_seqs,Mean_folding_init,Mean_folding_internal,Mean_folding_diff,Mean_folding_init_ribo,Mean_folding_internal_ribo,Mean_folding_diff_ribo
#Id,,,,,,,,,,,,,,,,,,,,,


In [23]:
###Any with ridiculously high / low nubmers of tRNA genes?
phylo_df[(phylo_df['tRNA_number'] < 20) | (phylo_df['tRNA_number'] > 200)]

,Organism,TaxonID,Full_Taxonomy,Banfield_tree_name,CDS_GC,CDS_GC_nostartstop,Ribo_prot_GC_nostartstop,CDS_number,Intragenic_GC,Gene_density,...,RP_information_zscore,aSD_binding_score_ribosome,aSD_binding_score_all,Internal_SD_seqs,Mean_folding_init,Mean_folding_internal,Mean_folding_diff,Mean_folding_init_ribo,Mean_folding_internal_ribo,Mean_folding_diff_ribo
#Id,,,,,,,,,,,,,,,,,,,,,


In [24]:
####Get rid of folks who abjectly failed any of the above tests
phylo_df = phylo_df.drop('GCA_000210415') #This guy is all NaN
phylo_df = phylo_df.drop('GCA_000093065') #Low CDS number
phylo_df = phylo_df.drop('GCA_000219175') #Low CDS number
phylo_df = phylo_df.drop('GCA_000441555') #Low CDS number
phylo_df = phylo_df.drop('GCA_000210735') #No 16s copies found
phylo_df = phylo_df.drop('GCA_000209815') #No 16s copies found

**Add in some summary statistics**

This should be a function since it's a copy/paste of the corresponding cell in growth rates with `growth_df` changed to `phylo_df`

In [25]:

###aSD binding score is the ribosomal protein score relative to all genes
phylo_df['aSD_binding_summary'] = (phylo_df['aSD_binding_score_ribosome'] - phylo_df['aSD_binding_score_all']) / phylo_df['aSD_binding_score_all']

phylo_df['folding_energy_init_summary'] = (phylo_df['Mean_folding_init_ribo'] - phylo_df['Mean_folding_init'])/ phylo_df['Mean_folding_init']
phylo_df['folding_energy_diff_summary'] = (phylo_df['Mean_folding_diff_ribo'] - phylo_df['Mean_folding_diff'])/ phylo_df['Mean_folding_diff']

###Codon usage bias score is the ribosomal protein score relative to all genes
###Using two methods that perform well (ours=info, novembre=method developed by John Novembre paper)
phylo_df['NC_summary_info'] = (phylo_df['Ribo_NC_info'] - phylo_df['All_NC_info']) / phylo_df['All_NC_info']
phylo_df['NC_summary_novembre'] = (phylo_df['Ribo_NC_novembre'] - phylo_df['All_NC_novembre']) / phylo_df['All_NC_novembre']

###Delta I score for all genes and ribosomal protein coding genes
phylo_df['SD_summary_all_info'] = phylo_df['Information_content_all(-20_-4)'] - phylo_df['Information_content_all_mean_random(-20_-4)']
phylo_df['SD_summary_ribo_info'] = phylo_df['Information_content_ribo(-20_-4)'] - phylo_df['Information_content_ribo_mean_random(-20_-4)']

###Summary score for "Traditional" SD methods, SD gene fraction minus expected SD gene fraction
phylo_df['SD_summary_all_motif_diff'] = (phylo_df['CDS_with_SD_utr_motif']/phylo_df['CDS_number']) - (phylo_df['Expected_CDS_with_SD_utr_motif']/phylo_df['CDS_number'])
phylo_df['SD_summary_all_binding_diff'] = (phylo_df['CDS_with_SD_utr_binding']/phylo_df['CDS_number']) - (phylo_df['Expected_CDS_with_SD_utr_binding']/phylo_df['CDS_number'])

**Manually inspect Counter to decide on some subsets to analyze**

In [26]:
phylo_df['Domain'] = phylo_df.Banfield_tree_name.apply(lambda x: x.split('_')[0])
phylo_df['Phylum'] = phylo_df.Banfield_tree_name.apply(lambda x: x.split('_')[1])
phylo_df['Class'] = phylo_df.Banfield_tree_name.apply(lambda x: x.split('_')[2])

In [30]:
print(Counter(phylo_df['Domain']))
print('#####')
print(Counter(phylo_df['Phylum']))

Counter({'Bacteria': 619, 'Archaea': 82})
#####
Counter({'Proteobacteria': 264, 'Firmicutes': 80, 'Actinobacteria': 78, 'Bacteroidetes': 63, 'Euryarchaeota': 56, 'Cyanobacteria': 29, 'Crenarchaeota': 20, 'Chlamydiae': 11, 'Chloroflexi': 10, 'Thermotogae': 8, 'Tenericutes': 8, 'Aquificae': 8, 'Spirochaetes': 7, 'Deinococcus': 6, 'FibrobacteresAcidobacteria': 6, 'Planctomycetes': 6, 'CPR': 5, 'Fusobacteria': 4, 'Thaumarchaeota': 4, 'Deferribacteres': 4, 'Synergistetes': 4, 'Thermodesulfobacteria': 3, 'Nitrospirae': 3, 'CP': 3, 'Elusimicrobia': 2, 'Dictyoglomi': 2, 'Armatimonadetes': 2, 'Gemmatimonadetes': 1, 'Nanoarchaeota': 1, 'Chrysiogenetes': 1, 'Caldiserica': 1, 'Korarchaeota': 1})


**Write some independent sets of data**

In [38]:
def trim_via_clade(df, tree):
    '''
    Restrictions need to be a tuple/list with 2 elements:
    
    (Column_name_to_select, Value_to_select_for_that_column)
    
    '''
    restricted_tree = copy.deepcopy(tree)
    restricted_df = df.copy(deep=True)
    
    names_list = list(restricted_df['Banfield_tree_name'])

    print('Leaves in tree (unique names)', len([i.name for i in restricted_tree.get_terminals()]),\
          len(list(set([i.name for i in restricted_tree.get_terminals()]))))

    ###Get rid of any entries in the tree that aren't in the dataframe
    for leaf in restricted_tree.get_terminals():
        if leaf.name not in names_list:
            restricted_tree.prune(leaf.name)
            
    ###Re-name the leaf names so that they match dataframe indices
    for leaf in restricted_tree.get_terminals():
        old_name = str(leaf.name)
        for i in restricted_df.index:
            if restricted_df.loc[i]['Banfield_tree_name'] == old_name:
                leaf.name = i
        if old_name == leaf.name:
            print('Something buggy here, go back and investigate')

    print('New number of leaves in tree (unique names)', len([i.name for i in restricted_tree.get_terminals()]),\
          len(list(set([i.name for i in restricted_tree.get_terminals()]))))

    print('Entries in dataframe (uniquely named entries)', len(restricted_df.index),\
          len(list(set(list(restricted_df.index)))))

    ###Get rid of any entries in the dataframe that aren't in the tree now
    to_drop = []
    for i in restricted_df.index:
        if i not in [j.name for j in restricted_tree.get_terminals()]:
            to_drop.append(i)
    restricted_df = restricted_df.drop(to_drop)
    print('Final dataset size', len(restricted_df.index))

    return restricted_df, restricted_tree

In [39]:
###This is the newick tree directly from the iTol publication
tree = Phylo.read('../Data/nmicrobiol201648-s6.txt', 'newick')

###Based on looking at the above info here is what I decided to analyze independently
independent_sets = [('Domain', 'Bacteria'),\
                    ('Domain', 'Archaea'),\
                    ('Phylum', 'Euryarchaeota'),\
                    ('Phylum', 'Proteobacteria'),\
                    ('Phylum', 'Chlamydiae'),\
                    ('Phylum', 'Bacteroidetes'),\
                    ('Phylum', 'Firmicutes'),\
                    ('Phylum', 'Cyanobacteria'),\
                    ('Phylum', 'Actinobacteria')]


for restriction in independent_sets:
    print('#####{}'.format(restriction))
    temp_df = phylo_df[phylo_df[restriction[0]] == restriction[1]]
    temp_df, temp_tree = trim_via_clade(temp_df, tree)
    Phylo.write(temp_tree, '../Data/Pruned_tree_{}_{}_10_2017.newick'.format(restriction[0], restriction[1]), 'newick')
    temp_df.to_csv('../Data/Pruned_df_{}_{}_10_2017.csv'.format(restriction[0], restriction[1]), sep='\t')

#####('Domain', 'Bacteria')
Leaves in tree (unique names) 3083 3083
New number of leaves in tree (unique names) 613 613
Entries in dataframe (uniquely named entries) 619 619
Final dataset size 613
#####('Domain', 'Archaea')
Leaves in tree (unique names) 3083 3083
New number of leaves in tree (unique names) 82 82
Entries in dataframe (uniquely named entries) 82 82
Final dataset size 82
#####('Phylum', 'Euryarchaeota')
Leaves in tree (unique names) 3083 3083
New number of leaves in tree (unique names) 56 56
Entries in dataframe (uniquely named entries) 56 56
Final dataset size 56
#####('Phylum', 'Proteobacteria')
Leaves in tree (unique names) 3083 3083
New number of leaves in tree (unique names) 264 264
Entries in dataframe (uniquely named entries) 264 264
Final dataset size 264
#####('Phylum', 'Chlamydiae')
Leaves in tree (unique names) 3083 3083
New number of leaves in tree (unique names) 11 11
Entries in dataframe (uniquely named entries) 11 11
Final dataset size 11
#####('Phylum', 'B

# Merging temperature dataset

In [42]:
####Two Different levels of stringency
df_phenotypes = pd.read_csv('../Data/ProTraits_binaryIntegratedPr0.90.txt', sep='\t')
# df_phenotypes = pd.read_csv('../Data/ProTraits_binaryIntegratedPr0.95.txt', sep='\t')

###Convert names
df_phenotypes['Tax_ID'] = df_phenotypes['Tax_ID'].astype(str)

In [43]:
df = pd.read_table('../Data/Pruned_df_Domain_Bacteria_10_2017.csv', sep='\t')
df['TaxonID'] = df['TaxonID'].astype(str)
print(len(df.index))

613


**Rather sloppily (this could/should(?) be done more elegantly as a merge)  get the taxon id's that are matched between the two df's and create a new column in the phenotype df to put these matching ones in**

In [44]:
df_phenotypes['New_TaxonID'] = ''

df_phenotype_dicty = {}
for index in df_phenotypes.index:
    df_phenotype_dicty[str(df_phenotypes.loc[index]['Tax_ID'])] = df_phenotypes.loc[index]['Organism_name']

df_dicty = {}
for index in df.index:
    df_dicty[str(df.loc[index]['TaxonID'])] = df.loc[index]['Organism']

    
hits = []
for i in df_dicty.keys():
    if i in df_phenotype_dicty.keys():
        hits.append(i)

for index in df_phenotypes.index:
    if df_phenotypes.loc[index]['Tax_ID'] in hits:
        df_phenotypes.set_value(index, 'New_TaxonID', df_phenotypes.loc[index]['Tax_ID'])

In [45]:
temp_df = pd.merge(df, df_phenotypes, how='inner', left_on='TaxonID', right_on='New_TaxonID')
print(len(temp_df.index))

57


**Dig deeper matching Genus/Species/XXX pairs**

In [46]:
listy = []
for i in list(df_dicty.values()):
    if len(i.split(' ')) >= 3:
        listy.append(' '.join(i.split(' ')[:3]))
counter_dict = Counter(listy)


df_dicty_rev = {}
for index in df.index:
    if len(df.loc[index]['Organism'].split(' ')) >= 3:
        if counter_dict[' '.join(df.loc[index]['Organism'].split(' ')[:3])] == 1:
            df_dicty_rev[' '.join(df.loc[index]['Organism'].split(' ')[:3])] = str(df.loc[index]['TaxonID'])

for trunc_name in list(counter_dict.keys()):
    if counter_dict[trunc_name] == 1 and '(' not in trunc_name:
        found = []
        if len(df_phenotypes[df_phenotypes['Organism_name'].str.contains(trunc_name)].index) > 0:
            for index in df_phenotypes[df_phenotypes['Organism_name'].str.contains(trunc_name)].index:
                if trunc_name in df_phenotypes.loc[index]['Organism_name']:
                    found.append(index)
            if len(found) == 1 and df_phenotypes.loc[found[0]]['New_TaxonID'] == '':
                df_phenotypes.set_value(found[0], 'New_TaxonID', str(df_dicty_rev[trunc_name]))

In [47]:
temp_df = pd.merge(df, df_phenotypes, how='inner', left_on='TaxonID', right_on='New_TaxonID')
print(len(temp_df.index))

74


**Now just grab Genus/Species pairs**

In [49]:
listy = []
for i in list(df_dicty.values()):
    listy.append(' '.join(i.split(' ')[:2]))
counter_dict = Counter(listy)


df_dicty_rev = {}
for index in df.index:
    if counter_dict[' '.join(df.loc[index]['Organism'].split(' ')[:2])] == 1:
        df_dicty_rev[' '.join(df.loc[index]['Organism'].split(' ')[:2])] = str(df.loc[index]['TaxonID'])

for trunc_name in list(counter_dict.keys()):
    if counter_dict[trunc_name] == 1:
        found = []
        for index in df_phenotypes[df_phenotypes['Organism_name'].str.contains(trunc_name)].index:
            if trunc_name in df_phenotypes.loc[index]['Organism_name']:
                found.append(index)
        if len(found) == 1 and df_phenotypes.loc[found[0]]['New_TaxonID'] == '':
            df_phenotypes.set_value(found[0], 'New_TaxonID', str(df_dicty_rev[trunc_name]))

In [50]:
temp_df = pd.merge(df, df_phenotypes, how='inner', left_on='TaxonID', right_on='New_TaxonID')
print(len(temp_df.index))

549


**Call it a solid days work and merge everything**

In [52]:
combined_df = pd.merge(df, df_phenotypes, how='inner', left_on='TaxonID', right_on='New_TaxonID')
combined_df.set_index(combined_df['#Id'], inplace=True)
combined_df.drop('#Id', axis=1, inplace=True)

In [53]:
print(len(df_phenotypes.index), len(df.index), len(combined_df.index))

3046 613 549


**Get the temperature variables**

In [55]:
print(len(combined_df[combined_df['temperaturerange=mesophilic'] == '1'].index))
print(len(combined_df[combined_df['temperaturerange=thermophilic'] == '1'].index))
temp_df = combined_df[(combined_df['temperaturerange=mesophilic'] == '1') ^ (combined_df['temperaturerange=thermophilic'] == '1')].copy()
temp_df['temperature'] = ''
for index in temp_df.index:
    if temp_df.loc[index]['temperaturerange=mesophilic'] == '1':
        temp_df.set_value(index, 'temperature', 'mesophiles')
    elif temp_df.loc[index]['temperaturerange=thermophilic'] == '1':
        temp_df.set_value(index, 'temperature', 'thermophiles')
    else:
        print("Well, this is certainly an unexpected occurrence")

print(Counter(temp_df['temperature']))

temp_df.to_csv('../Data/Pruned_df_Domain_Bacteria_10_2017_WITH_PROTRAITS_TEMPERATURES.csv', sep='\t')

413
68
Counter({'mesophiles': 413, 'thermophiles': 68})


**Habitat variables**

In [56]:
print(len(combined_df[combined_df['habitat=freeliving'] == '1'].index))
print(len(combined_df[combined_df['habitat=hostassociated'] == '1'].index))
temp_df = combined_df[(combined_df['habitat=freeliving'] == '1') ^ (combined_df['habitat=hostassociated'] == '1')].copy()

temp_df['habitat'] = ''
for index in temp_df.index:
    if temp_df.loc[index]['habitat=freeliving'] == '1':
        temp_df.set_value(index, 'habitat', 'free')
    elif temp_df.loc[index]['habitat=hostassociated'] == '1':
        temp_df.set_value(index, 'habitat', 'host')

print(Counter(temp_df['habitat']))


temp_df.to_csv('../Data/Pruned_df_Domain_Bacteria_10_2017_WITH_PROTRAITS_HABITAT.csv', sep='\t')

356
114
Counter({'free': 353, 'host': 111})


In [57]:
print(len(combined_df[combined_df['pathogenic_in_fish'] == '1'].index))
print(len(combined_df[combined_df['pathogenic_in_mammals'] == '1'].index))
print(len(combined_df[combined_df['pathogenic_in_plants'] == '1'].index))

temp_df = combined_df
temp_df['pathogen'] = ''
for index in temp_df.index:
    listy = [temp_df.loc[index]['pathogenic_in_fish'],\
        temp_df.loc[index]['pathogenic_in_mammals'],\
        temp_df.loc[index]['pathogenic_in_plants']]
    if listy.count('?') == 3:
        temp_df.set_value(index, 'pathogen', '?')
    elif listy.count('1') > 0:
        temp_df.set_value(index, 'pathogen', 'Yes')
    else:
        temp_df.set_value(index, 'pathogen', 'No')

temp_df = temp_df[temp_df['pathogen'] != '?']
print(len(temp_df.index))
print(Counter(temp_df['pathogen']))
temp_df.to_csv('../Data/Pruned_df_Domain_Bacteria_10_2017_WITH_PROTRAITS_PATHOGEN.csv', sep='\t')

1
75
12
499
Counter({'No': 411, 'Yes': 88})


In [61]:
for i in combined_df.columns:
    if i.find('motil') != -1:
        print(i)

motility
(sporeform,gramposit,nonmotil,motil,rodshap)
(glide,motil,motor,pigment,pathogen)
(nonpathogen,blight,fire,shoot,motil)
(virul,pathogen,highli,econom,motil)
(anox,sediment,ground,nonmotil,reclassifi)
(bioluminesc,gramneg,nonmotil,reliabl,biodegrad)
(faec,nonsporeform,nonmotil,gramneg,rodshap)
(gramposit,motil,sporeform,whippl,multisystem)


In [66]:
Counter(combined_df['mobility'])

Counter({'0': 9, '1': 12, '?': 528})

In [64]:
combined_df['motility']

#Id
GCA_000010525    ?
GCA_000143885    0
GCA_000503875    ?
GCA_000695975    ?
GCA_000009085    ?
GCA_000430105    ?
GCA_000013805    1
GCA_000016685    0
GCA_000015965    ?
GCA_000465255    0
GCA_000010245    1
GCA_000009605    ?
GCA_000348585    1
GCA_000007325    0
GCA_000359505    ?
GCA_000011945    0
GCA_000025985    0
GCA_000014085    ?
GCA_000632905    ?
GCA_000012805    ?
GCA_000009125    ?
GCA_000013745    ?
GCA_000063605    0
GCA_000238175    ?
GCA_000010085    ?
GCA_000010045    ?
GCA_000014385    0
GCA_000019885    ?
GCA_000018265    0
GCA_000018665    ?
                ..
GCA_000739375    ?
GCA_000740455    ?
GCA_000743955    1
GCA_000750195    ?
GCA_000750515    0
GCA_000754275    ?
GCA_000310245    ?
GCA_000597785    ?
GCA_000495935    ?
GCA_000224005    ?
GCA_000243115    ?
GCA_000242935    ?
GCA_000286435    ?
GCA_000328765    ?
GCA_000146025    ?
GCA_000973625    ?
GCA_000340565    ?
GCA_001042695    0
GCA_001042675    0
GCA_000508285    ?
GCA_000723785    ?
GCA_0003